In [1]:
import os

In [ ]:
%pwd

In [3]:
os.chdir("../")

In [ ]:
%pwd

----------------------------------- entity ------------------------------------------

In [5]:
from dataclasses import dataclass
from pathlib import Path

# Clase para definir la configuración para entrenar un modelo
# y los objetos de la clase sean inmutables, es decir, que 
# no se puedan modificar después de ser creados.
@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

----------------------------------- configuration manager ------------------------------------------

In [6]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [7]:
# Crear una clase para administrar las configuraciones del modelo a entrenar
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_column = schema.name
            
        )

        return model_trainer_config

----------------------------------- components ------------------------------------------

* ----------------------------------- model_trainer.py ------------------------------------------

In [8]:
import pandas as pd
import os
from mlProject import logger
from sklearn.linear_model import ElasticNet
import joblib

In [9]:
# Crear una clase para entrenar el modelo
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    
    def train(self):
        # Cargar el dataset
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        # establer la variable "X" y "y" tanto en el dataset de entrenamient y test
        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]

        # Crear el modelo 
        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        # Entrenar el modelo
        lr.fit(train_x, train_y)
        # Guardar el modelo entrenado en la ruta establecida(dentro de artifacts)
        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))


----------------------------------- pipeline ------------------------------------------

* ----------------------------------- stage_04_model_trainer_pipeline ------------------------------------------

In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2023-07-29 16:55:19,151: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-07-29 16:55:19,171: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-29 16:55:19,180: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-07-29 16:55:19,184: INFO: common: created directory at: artifacts]
[2023-07-29 16:55:19,188: INFO: common: created directory at: artifacts/model_trainer]
